In [ ]:
from salishsea_tools import timeseries_tools, nc_tools
import numpy as np
import xarray as xr
import resource
from time import time

In [ ]:
# Get file names
timerange = ['2016 Dec 1 00:00', '2016 Dec 30 23:00']
filesT = timeseries_tools.make_filename_list(timerange, 'T', model='nowcast-blue', resolution='h')

# Load mask for passing to load_NEMO_timeseries
mask = xr.open_dataset('/home/mdunphy/MEOPAR/NEMO-forcing/grid/mesh_mask_downbyone2.nc')

In [ ]:
# Select one of the cases to run (restart the kernel each time to get accurate memory usage)
fun = 'scDataset'
#fun = 'load_NEMO_timeseries'
#fun = 'xarray'

In [ ]:
# Record start time
t0 = time()

if fun == 'scDataset':
    with nc_tools.scDataset(filesT) as ds:
        ts  = ds.variables['votemper'][:,:,450,:]

if fun == 'load_NEMO_timeseries':
    T_450, coords = timeseries_tools.load_NEMO_timeseries(filesT, mask, 'votemper', 'y', index=450)

if fun == 'xarray':
    with xr.open_mfdataset(filesT) as ds:
        tx = ds.variables['votemper'][:,:,450,:].values    

mem = resource.getrusage(resource.RUSAGE_SELF).ru_maxrss
print("Loaded in {:.2f}s, Peak Memory Usage {:.2f} MB ({})".format(time()-t0, mem/1024.0, fun))

In [ ]:
# Benchmark with three days worth of files (72 slices), warmed file cache
#  Loaded in 4.17s, Peak Memory Usage 165.80 MB (scDataset)
#  Loaded in 4.43s, Peak Memory Usage 570.95 MB (load_NEMO_timeseries)
#  Loaded in 18.02s, Peak Memory Usage 9390.45 MB (xarray)

# Benchmark with 30 days worth of files (720 slices), warmed file cache
#  Loaded in 38.60s, Peak Memory Usage 373.03 MB (scDataset)
#  Loaded in 40.76s, Peak Memory Usage 571.10 MB (load_NEMO_timeseries)
#  xarray failed (ran out of memory, kernel crashed....)
